Creates the database of the toolings by extracting the output file using RyR Generator target files

In [1]:
#% pip install openpyxl
import os, re, configparser
import pandas as pd
import numpy as np
from globals import glob
from utilities_database import prepare_data, prepare_database, consult_database, clear_databases, retrieve_data, rename_index, get_date, get_sigma, rename_limits_table, show_table
from utilities_analysis import mean_calculator, limits_generator, ini_generator, RyR, z_score_filter, reset_df
from utilities_plotting import plot_scatter, plot_capability
from utilities_widgets import widget_select_one
import _db_tools as db

Database *input.db* found in: c:\Users\lucio\Desktop\Codebase\Python_Eiit_RyR\a2_Analyser\database\input.db
Database *output.db* found in: c:\Users\lucio\Desktop\Codebase\Python_Eiit_RyR\a2_Analyser\database\output.db


In [21]:
##File filtering
extension = "xlsx"
file_list = os.listdir("../a1_input/")
filtered_list = [filename for filename in file_list if filename.endswith(extension)]
name = next((string for string in filtered_list if glob.tooling in string), None)
selected_button = widget_select_one(options=filtered_list, value=name, layout={'width': '600px'})
print("Files ready for storage:")
display(selected_button)


Files ready for storage:


Select(description='Files:', index=1, layout=Layout(width='600px'), options=('TOP_PASSAT_B9_2023y-12m-13d_14h-…

In [22]:
##Database generation
confirmation = input("You are going to modify the database. Are you sure you wrote the correct tooling in the globals file? (y/n): ").strip().lower()
if confirmation == 'y':
    try:
        file = selected_button.value 
        filepath = os.path.join(os.path.abspath("../a1_input/"), file)
    except:
        print("No selected file. Check the selection box.")
        raise SystemExit
    df = prepare_data(filepath)
    MEAS = prepare_data(filepath, filter="measurements") #Load the output from RyR_Generator into a df
    LIMITS = prepare_data(filepath, filter="limits") #Load the output from RyR_Generator into a df
    file, _ = os.path.splitext(file)
    file = re.sub(r'\W', '_', file)
    prepare_database("input.db", MEAS, file+"_measurements") #Store a df for the measures inside the database of the project
    prepare_database("input.db", LIMITS, file+"_limits") #Store a df for the limits inside the database of the project
    measurements_file = rename_limits_table("input.db", file+"_measurements")
    limits_file = rename_limits_table("input.db", file+"_limits")
    print("Files renamed for clarity.")
    table_names = consult_database("input.db", verbose=True)
else:
    print("Operation canceled.")

Dataframe stored as *VA_007_2023y_12m_21d_14h_05m_01s_measurements*
Dataframe stored as *VA_007_2023y_12m_21d_14h_05m_01s_limits*
Files renamed for clarity.
*input.db* actual contents:
    VA_007_limits_2023y_12m_21d_14h_05m_01s
    VA_007_measurements_2023y_12m_21d_14h_05m_01s


In [23]:
##Database management
try:
    print("Latest tables added: ")
    for table in table_names:
        print(f"    {table}")
except Exception as e:
    consult_database("input.db")

Latest tables added: 
    VA_007_limits_2023y_12m_21d_14h_05m_01s
    VA_007_measurements_2023y_12m_21d_14h_05m_01s


In [24]:
##Add tooling data
name = re.sub(r'\W', '_', glob.tooling + "_" + get_date()) + "_fixture&data"
confirmation = input("You are going to modify the database. Are you sure you wrote the correct tooling in the globals file? (y/n): ").strip().lower()
if confirmation == 'y':
    FIXTURE_DATA = pd.DataFrame(data=[glob.tooling, glob.lenses_per_nest, glob.nests_number,
                                glob.x_tolerance, glob.y_tolerance, glob.lo_tolerance,
                                glob.hi_tolerance], index=["Tooling name", "Lenses per nest",
                                "number of nests", "x-axis tolerance", "y-axis tolerance", "lower tolerance",
                                "higher tolerance"])
    FIXTURE_DATA.rename(columns={0:"Tooling data"}, inplace=True)
    prepare_database("input.db", FIXTURE_DATA, name, add_index=True) #Store a df for the measures inside the database of the project
    fixture_data = rename_limits_table("input.db", name)
    print("Files renamed for clarity.")
    table_names = consult_database("input.db", verbose=True)
else:
    print("Operation canceled.")

Dataframe stored as *VA_007_2023y_12m_28d_13h_46m_12s_fixture_data*
Files renamed for clarity.
*input.db* actual contents:
    VA_007_fixture_data_2023y_12m_28d_13h_46m_12s
    VA_007_limits_2023y_12m_21d_14h_05m_01s
    VA_007_measurements_2023y_12m_21d_14h_05m_01s


In [25]:
###WARNING###
clear_databases(["input.db", "output.db"])

In [2]:
show_table("input.db", "VA_007_fixture_data_2023y_12m_28d_13h_46m_12s")

table 1: VA_007_fixture_data_2023y_12m_28d_13h_46m_12s
    Rows: []
    Columns: 1
Columns name: ('Tooling data',)
    ('VA-007',)
    ('3',)
    ('2',)
    ('0.0125',)
    ('0.0165',)
    ('0.02',)
    ('0.03',)
